# <center> RAG для нормативно правовых актов </center>

In [1]:
import os

import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import torch

from pymilvus import MilvusClient

from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import HuggingFaceEndpoint


os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

### Функции для подготовки данных для хранилища

Функции для создания эмбеддингов. Надо заняться тем, чтобы сделать на сервере эмбеддинг каждого акта при помощи mistral e5

In [2]:
# For embedding model multilingual-e5-base

def average_pool(
    last_hidden_states: Tensor,
     attention_mask: Tensor
) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def make_embedding(input_texts, model, tokenizer):
    batch_dict = tokenizer(
        input_texts, max_length=512, 
        padding=True, truncation=True, 
        return_tensors='pt'
    )
    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings.tolist()

В milvus можно вставлять данные как список словарей. В каждом словаре в поле "vector" лежит эмбединг, потом по этому полю будет идти поиск. Остальные поля это мета данные. Тут я пока не подогла поля под нашу задачу

In [3]:
def converting_data(embeds, summurys, original_texts, id_begin=0):
    data = []

    for i, (embed, summury, original_text) in enumerate(zip(embeds, summurys, original_texts)):
        data.append({
            'id': id_begin + i, 
            'vector': embed, 
            'summury': summury, 
            'original_text': original_text,  # TODO: add meta data
        })
    
    return data

In [4]:
system_prompt = """
    Ты разговариваешь только на русском языке. Ты - профессиональный юрист. За каждый правильный ответ ты будешь получать по 100$.
"""

question = """ 
    Ты разговариваешь только на русском языке. Ты специалист по юристпруденции. 
    Тебе на вход приходит нормативно правовой акт. 
    Суммаризируй его, найди основную суть документа. 
    Твой ответ должен быть кратким, в ответе просто напиши саммари без лишних вводных фраз. 
    Особое внимание удели числам, процентам, датам и внесенным изменениям.
    Нормативно-правовой акт:
    {context}
"""

In [5]:
from llamaapi import LlamaAPI

def conntect_to_llama_by_api(llama_token):
    llama = LlamaAPI(llama_token)
    return llama

def get_answer_by_api(context, llm, system_prompt, question, max_tokens=4096,temperature=0.1):
    api_request_json = {
        'model': 'llama3.1-8b',
        'max_tokens': max_tokens,
        'temperature': temperature,
        "messages": [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': question.format(context=context)}
        ]
    }
    response = llm.run(api_request_json)
    return response.json()['choices'][0]['message']['content']
    

In [6]:
from tqdm import tqdm

def make_summary(docs, llm, system_prompt, question, get_answer_func=get_answer_by_api):
    summarys = []
    for doc in tqdm(docs):
        summarys.append(get_answer_func(doc, llm, system_prompt, question))
    return summarys

### Подготовка данных для хранилища

In [7]:
data_path = ''  # path to dataset

with open(data_path, encoding='utf8') as file:
    txt = file.read()

list_docs = txt.split('\n')
list_docs = [doc for doc in list_docs if doc]

In [14]:
summarys = make_summary(list_docs[:10], llama, system_prompt, question)  # TODO: use full data

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:26<00:00,  2.65s/it]


In [12]:
token = ''  # llamaapi token
llama = conntect_to_llama_by_api(token)
get_answer_by_api(doc, llama, system_prompt, question)

'В постановлении Губернатора Ханты-Мансийского автономного округа-Югры от 28.12.2017 № 139 вносятся изменения в Инструкцию по делопроизводству в государственных органах Ханты-Мансийского автономного округа-Югры и исполнительных органах государственной власти Ханты-Мансийского автономного округа-Югры. \n\nИзменения касаются следующих разделов:\n\n- раздела I: заменены ссылки на ГОСТ Р 6.30-2003 на ГОСТ Р 7.0.97-2016;\n- раздела III: изменены требования к оформлению реквизита "Адресат", добавлены правила для адресования документов физическим лицам, а также правила для отправки документов по электронной почте или по факсимильной связи;\n- раздела IV: изменены правила хранения и регистрации документов, добавлены правила для регистрации документов в СЭД;\n- раздела VII: добавлены правила для согласования обращений в федеральные органы государственной власти;\n- раздела X: заменены ссылки на ГОСТ Р 6.30-2003 на ГОСТ Р 7.0.97-2016.\n\nИзменения вступают в силу с 1 июля 2018 года.'

In [8]:
doc = list_docs[0]
doc[:500]

'ПОСТАНОВЛЕНИЕ ГУБЕРНАТОРА ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА-ЮГРЫ от 28.12.2017 № 139.  О ВНЕСЕНИИ ИЗМЕНЕНИЙ В ПРИЛОЖЕНИЕ К ПОСТАНОВЛЕНИЮ ГУБЕРНАТОРА ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА – ЮГРЫ ОТ 30 ДЕКАБРЯ 2012 ГОДА N 176 "ОБ ИНСТРУКЦИИ ПО ДЕЛОПРОИЗВОДСТВУ В ГОСУДАРСТВЕННЫХ ОРГАНАХ ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ И ИСПОЛНИТЕЛЬНЫХ ОРГАНАХ ГОСУДАРСТВЕННОЙ ВЛАСТИ ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ" ГУБЕРНАТОР ХАНТЫ-МАНСИЙСКОГО АВТОНОМНОГО ОКРУГА - ЮГРЫ ПОСТАНОВЛЕНИЕ от 28 д'

In [20]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

embeddings = make_embedding(summarys, model, tokenizer)
embedding_dim = len(embeddings[0])
insert_data = converting_data(embeddings, summarys, list_docs)

### Создание коллекции данных

In [22]:
client = MilvusClient(uri="./milvus_demo.db")

collection_name = "acts"

In [23]:
client.has_collection(collection_name=collection_name)

True

In [ ]:
# удаление коллекции

# if client.has_collection(collection_name=collection_name):
#     client.drop_collection(collection_name=collection_name)

In [24]:
# Создание колекции данных
client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="COSINE"  # Inner product distance
)

# Вставка данных в коллекцию
client.insert(collection_name=collection_name, data=insert_data)

{'insert_count': 10, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'cost': 0}

## Поиск по коллекции

Нужно добавить переформулировку запроса перед тем как делать из него эмбеддинг

In [36]:
# можно делать запрос на переформулировку

quary = 'Кто явяется губернатором ХМАО?'

embedding_quary = make_embedding(quary, model, tokenizer)

### Поиск топ 5 подходящих документов

In [37]:
search_res = client.search(
    collection_name=collection_name,
    data=embedding_quary, 
    limit=5,
    search_params={"metric_type": "COSINE", "params": {}}, 
    output_fields=["summary"],  # Return the text field
)

In [38]:
search_res

data: ["[{'id': 3, 'distance': 0.7847653031349182, 'entity': {}}, {'id': 1, 'distance': 0.7771220803260803, 'entity': {}}, {'id': 2, 'distance': 0.7715886235237122, 'entity': {}}, {'id': 4, 'distance': 0.7688708305358887, 'entity': {}}]"] 

In [40]:
search_res[0][0]['id']

3

In [102]:
retrieved_lines_with_distances = [
    (res["id"]["summary"], res["distance"]) for res in search_res[0]
]

for text, res in retrieved_lines_with_distances:
    print(name, '\n')
    print('Близость =', res, '\n\n')

KeyError: 'summary'

### Генерация ответа LLM

подтянем llm c HuggingFace, для этого в строке импортов введи свой токен с HuggingFace

In [ ]:
system_prompt = """
    Ты разговариваешь только на русском языке. Ты - профессиональный юрист. За каждый правильный ответ ты будешь получать по 100$.
"""

question = """ 
    Ты разговариваешь только на русском языке. Ответь на вопрос, опираясь на нормативно-правовые акты из контектса
    Контекст: {context}
"""
llm = conntect_to_llama_by_api(token)

# TODO: сделать подгрузку контекста из БД
# TODO: протестировать качество ответа

print(get_answer_by_api(context, llm, system_prompt, question, max_tokens=4096,temperature=0.5))
